In [ ]:
import os
import os.path as osp
import pandas as pd
import json
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
from utils import display_target_distractors, parse_filename
sys.path.append('./..')
from configuration import Config
from datasets.utils import get_refcoco_df

In [ ]:
split = 'val'

In [ ]:
# prepare config and refcoco df

config = Config()

refcoco_df = get_refcoco_df(config.ref_dir)

refcoco_df = refcoco_df.reset_index().groupby('ann_id').agg({
    'sent_id': list, 'caption': list, 
    'ref_id': 'first', 'refcoco_split': 'first', 
    'coco_split': 'first', 'image_id': 'first', 
    'bbox': 'first'
}).reset_index()

In [ ]:
# get available files

data_path = osp.abspath('./../data/results/')

metric_files = sorted([f for f in os.listdir(data_path) if f.endswith('eval.json')])
expression_files = sorted([f for f in os.listdir(data_path) if f.endswith('generated.json')])

In [ ]:
# aggregate quantitative results

series = []

condition, epoch, split, noise_level = parse_filename(metric_files[1])
for filename in metric_files:
    condition, epoch, split, noise_level = parse_filename(filename)
    
    with open(osp.join(data_path, filename)) as f:
        data = json.load(f)

    s = pd.Series({
        'condition': condition,
        'epoch': epoch,
        'split': split,
        'noise_level': noise_level,
        'Bleu_1': data['Bleu_1'],
        'Bleu_2': data['Bleu_2'],
        'Bleu_3': data['Bleu_3'],
        'Bleu_4': data['Bleu_4'],
        'METEOR': data['METEOR'],
        'ROUGE_L': data['ROUGE_L'],
        'CIDEr': data['CIDEr'],
    })

    series.append(s)

metrics_df = pd.concat(series, axis=1).T
metrics_df.index = metrics_df.apply(lambda x: f'{x.condition}_{x.noise_level}', axis=1)

In [ ]:
# plot CIDEr scores

for condition in ['loc', 'loc_glob']:
    metrics_df.loc[metrics_df.condition==condition].sort_values(by='noise_level').set_index('noise_level')['CIDEr'].plot(label=condition)
plt.grid(linestyle='dotted')
plt.xticks(sorted(pd.unique(metrics_df.noise_level)))
plt.xlabel('Noise Level')
plt.ylabel('CIDEr')
plt.legend()

In [ ]:
# aggregate generated expressions for refcoco annotation ids

expressions = defaultdict(dict)

for file in expression_files:

    fpath = osp.join(data_path, file)
    condition, _, _, noise_level = parse_filename(file)
    condition_key = f'{condition}_{noise_level}'

    with open(fpath) as f:
        this_file_data = json.load(f)

    for entry in this_file_data:
        expressions[entry['ann_id']][condition_key] = entry['expression']

In [ ]:
sample = refcoco_df.loc[refcoco_df.refcoco_split == split].sample(100,replace=False,random_state=42)

In [ ]:
for i in range(len(sample)):
      entry = sample.iloc[i]

      idx = entry.name
      ann_id = entry.ann_id
      anns = entry.caption

      print(f'ann_id: {ann_id}', 
            'Annotations:', 
            *['\t'+ann for ann in anns], 
            sep='\n')

      display_target_distractors(entry_id=idx, refcoco_data=refcoco_df, coco_base=config.dir)

      for condition in sorted(expressions[ann_id].keys()):
            print(f'{condition.ljust(14)}: {expressions[ann_id][condition]}')

      print('\n---------\n')